In [8]:
# default_exp motor

In [3]:
from private import server_vars

In [4]:
base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

In [23]:
# export
import requests

class Motor:
    "Wrapper for Alteryx Server API."
    
    def __init__(self, 
                 base_url: str, 
                 login_email: str, 
                 login_pwd: str):
        
        self.base_url = base_url
        self.auth_url = f"{self.base_url}api/v1/authenticate"
        self.login_email = login_email
        self.login_pwd = login_pwd
        
        self.headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate"
        }
        
        self.jar = requests.cookies.RequestsCookieJar()
        self.is_authenticated = False
        self.authenticate()
        
        
    def authenticate(self) -> requests.Response:
        payload = {
            "email": self.login_email, 
            "password": self.login_pwd
        }
        response = requests.post(url=self.auth_url, 
                                 json=payload, 
                                 cookies=self.jar, 
                                 verify=False, 
                                 headers=self.headers)
        if response.status_code == 200:
            self.jar.update(response.cookies)
            self.is_authenticated = True
            return response
        
    def get_jobs(self):
        response = requests.get(f"{self.base_url}api/v1/jobs",
                     cookies=self.jar,
                     verify=False)
        return response

In [24]:
motor = Motor(base_url,
              login_email, 
              login_pwd)

c:\users\jkabler\.conda\envs\yx_motor\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'test.aep-mono.devops.alteryx.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [25]:
motor.is_authenticated

True

In [26]:
motor.get_jobs()

c:\users\jkabler\.conda\envs\yx_motor\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'test.aep-mono.devops.alteryx.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<Response [200]>

In [13]:
response1.status_code

200

In [47]:
jar['ayxSession']

's%3Ac133e377-8a73-4a74-a6ad-2eae74d8cd9c.CAw5B4uubIQpCzpEB5b0aWKd%2F52GfuH2E3o0ix%2FXldA'